In [1]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import ShuffleSplit, cross_val_score

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/madson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
output_path = "../data/interim/news.csv"

df = pd.read_csv(output_path)

In [3]:
corpus = df.text.to_list()
labels = df.label.replace({"true": 1, "fake": 0})

In [4]:
vectorizer = CountVectorizer(
    stop_words = nltk.corpus.stopwords.words('portuguese'),
    max_features = 1000
)
pca = TruncatedSVD(500)
scaler = StandardScaler()
model = SVC()

In [5]:
pipeline = Pipeline(steps=[
    ("vectorizer", vectorizer),
    ("pca", pca),
    ("normalize", scaler),
    ("model", model)
])

split = ShuffleSplit(n_splits=30, test_size=.2)
scores = cross_val_score(pipeline, corpus, labels, cv=split)

In [6]:
split = ShuffleSplit(n_splits=30, test_size=.2)
scores = cross_val_score(pipeline, corpus, labels, cv=split)

In [8]:
scores.mean()

0.9510185185185184

In [9]:
scores.std()

0.0051430580351927825